In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data = pd.read_csv('Datasets\Project 19 Breast Cancer Data.csv')


In [ ]:
data.head()


In [ ]:
data.info()


In [ ]:
import seaborn as sns


In [ ]:
sns.set_style('whitegrid')
sns.countplot(x = 'diagnosis', data = data)

In [ ]:
dataset = data
dataset['diagnosis'].replace(['M','B'], [1,0], inplace = True)

In [ ]:


dataset.drop('Unnamed: 32',axis = 1, inplace = True)



In [ ]:
corr = dataset.corr()
plt.figure(figsize = (25,25))
sns.heatmap(corr, annot = True)

In [ ]:
dataset.corr()


In [ ]:
dataset.drop(['id','symmetry_se','smoothness_se','texture_se','fractal_dimension_mean'], axis = 1, inplace = True)


In [ ]:
dataset.head()


In [ ]:
plt.figure(figsize = (25,25))
sns.heatmap(dataset.corr(), annot = True)


In [ ]:
X = dataset.drop('diagnosis', axis = 1)
y = dataset['diagnosis']

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [ ]:
print("Train Set: ", X_train.shape, y_train.shape)
print("Test Set: ", X_test.shape, y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score


In [ ]:
confusion_matrix(y_test, model.predict(X_test))


In [ ]:
print(f"Accuracy is {round(accuracy_score(y_test, model.predict(X_test))*100,2)}")


In [ ]:
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
classifier = RandomForestClassifier(n_jobs = -1)


In [ ]:
search_clfr = RandomizedSearchCV(classifier, param_distributions = param_dist, n_jobs=-1, n_iter = 40, cv = 9)


In [ ]:
search_clfr.fit(X_train, y_train)


In [ ]:
params = search_clfr.best_params_
score = search_clfr.best_score_
print(params)
print(score)


In [ ]:
claasifier=RandomForestClassifier(n_jobs=-1, n_estimators=200,bootstrap= True,criterion='gini',max_depth=20,max_features=8,min_samples_leaf= 1)


In [ ]:
classifier.fit(X_train, y_train)


In [ ]:
confusion_matrix(y_test, classifier.predict(X_test))


In [ ]:
print(f"Accuracy is {round(accuracy_score(y_test, classifier.predict(X_test))*100,2)}%")


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.datasets import load_breast_cancer

# Load dataset
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target

# Define features and target
X = df.drop(columns=['target'])
y = df['target']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize models
models = {
    "Logistic Regression": LogisticRegression(max_iter=5000),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "K-Nearest Neighbour": KNeighborsClassifier(n_neighbors=5)
}

# Train and evaluate models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    auc_roc = roc_auc_score(y_test, y_pred)
    results[name] = accuracy * 100
    print(f"{name} Accuracy: {accuracy * 100:.2f}%")
    print(f"{name} AUC-ROC: {auc_roc:.2f}")
    print(classification_report(y_test, y_pred))
    print("-" * 50)

# Check for Overfitting using Cross-Validation
for name, model in models.items():
    cv_scores = cross_val_score(model, X, y, cv=5)
    print(f"{name} Cross-validation Accuracy: {np.mean(cv_scores) * 100:.2f}%")

# Plot Model Performance
plt.figure(figsize=(8,5))
sns.barplot(x=list(results.keys()), y=list(results.values()))
plt.ylabel("Accuracy (%)")
plt.title("Model Comparison")
plt.show()

In [ ]:
import pickle
pickle.dump(classifier, open('cancer.pkl', 'wb'))